In [ ]:
from p2pfl.node import Node
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from collections import OrderedDict
import torch
import time

def test_convergence(x):
    n,r = x

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.1)

    # Check if they are connected
    for node in nodes:
        assert len(node.neightboors) == n-1

    # Start Learning
    nodes[0].set_start_learning(rounds=r,epochs=0)

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        for f in [node.round is None for node in nodes]:
            finish = finish and f

        if finish:
            break


    # Validamos Modelos obtenidos sean iguales
    model = None
    first = True
    for node in nodes:
        if first:
            model = node.learner.get_parameters()
            first = False
        else:
            for layer in model:
                a = torch.round(model[layer], decimals=2)
                b = torch.round(node.learner.get_parameters()[layer], decimals=2)
                assert torch.eq(a, b).all()

    # Cerrar
    for node in nodes:
        node.stop()
        time.sleep(.1) #Esperar por la asincronía
    
    return nodes
        
nodes = test_convergence((2,2))

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 63669
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 63670
INFO:root:Conexión aceptada con ('127.0.0.1', 63670)
INFO:root:Broadcasting start learning...
INFO:root:Sending Initial Model Weights
INFO:root:Initialicing Model Weights
INFO:root:Broadcasting model to all clients...
INFO:root:Broadcasting Number of Samples...
INFO:root:Model initialized
INFO:root:Training...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Broadcasting Number of Samples...
INFO:root:Training...
INFO:root:Model added (1/2)
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/2)
INFO:root:Agregating models.


Exception in thread Thread-12:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/learning/agregators/agregator.py", line 22, in run
    self.node.learner.set_parameters(self.agregate(self.models))
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/learning/agregators/fedavg.py", line 34, in agregate
    accum = (models[-1][0]).copy()
KeyError: -1


INFO:root:Model added (2/2)
INFO:root:Agregating models.


Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/learning/agregators/agregator.py", line 22, in run
    self.node.learner.set_parameters(self.agregate(self.models))
  File "/Users/pedro/Documents/UDC/4/TFG/federated_learning_p2p/p2pfl/learning/agregators/fedavg.py", line 34, in agregate
    accum = (models[-1][0]).copy()
KeyError: -1


In [1]:
from p2pfl.const import HEARTBEAT_FREC, TIEMOUT
from p2pfl.learning.pytorch.mnist_examples.mnistfederated_dm import MnistFederatedDM
from p2pfl.learning.pytorch.mnist_examples.models.cnn import CNN
from p2pfl.learning.pytorch.mnist_examples.models.mlp import MLP
from p2pfl.node import Node
import pytest
import time
        
def test_node_down_on_learning(n):

    # Node Creation
    nodes = []
    for i in range(n):
        node = Node(MLP(),MnistFederatedDM())
        node.start()
        nodes.append(node)

    # Node Connection
    for i in range(len(nodes)-1):
        nodes[i+1].connect_to(nodes[i].host,nodes[i].port)
        time.sleep(0.5)

    # Start Learning
    nodes[0].set_start_learning(rounds=2,epochs=0)

    # Stopping node
    nodes[1].stop()

    # Wait 4 results
    while True:
        time.sleep(1)
        finish = True
        x = [node.round is None for node in nodes]
        print(x)
        for f in x:
            finish = finish and f

        if finish:
            break

    for node in nodes:
        node.stop()

while True:
    test_node_down_on_learning(5)

Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 62747
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 62748
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 62749
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 62750
Train: 54000 Val:6000 Test:10000
INFO:root:Nodo a la escucha en 127.0.0.1 62751
INFO:root:Conexión aceptada con ('127.0.0.1', 62748)
INFO:root:Conexión aceptada con ('127.0.0.1', 62749)
INFO:root:Conexión aceptada con ('127.0.0.1', 62747)
INFO:root:Conexión aceptada con ('127.0.0.1', 62750)
INFO:root:Conexión aceptada con ('127.0.0.1', 62748)
INFO:root:Conexión aceptada con ('127.0.0.1', 62747)
INFO:root:Conexión aceptada con ('127.0.0.1', 62751)
INFO:root:Conexión aceptada con ('127.0.0.1', 62749)
INFO:root:Conexión aceptada con ('127.0.0.1', 62748)
INFO:root:Conexión aceptada con ('127.0.0.1', 62747)
INFO:root:Broadcasting start learning...
INFO:root:Initialicing Model 

INFO:root:Agregating models.
INFO:root:Agregating models.
INFO:root:Round 1 of 2 finished. (('127.0.0.1', 62783))
INFO:root:Training...
INFO:root:Round 1 of 2 finished. (('127.0.0.1', 62781))
INFO:root:Round 1 of 2 finished. (('127.0.0.1', 62782))
INFO:root:Model added (1/4)
INFO:root:Training...
INFO:root:Training...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (1/4)
INFO:root:Model added (1/4)
INFO:root:Broadcasting model to all clients...
INFO:root:Broadcasting model to all clients...
INFO:root:Model added (2/4)
INFO:root:Model added (2/4)
INFO:root:Model added (2/4)
INFO:root:Model added (3/4)
INFO:root:Model added (2/4)
INFO:root:Model added (3/4)
INFO:root:Model added (3/4)
INFO:root:Model added (4/4)
INFO:root:Model added (3/4)
INFO:root:Agregating models.
INFO:root:Round 2 of 2 finished. (('127.0.0.1', 62779))
INFO:root:Finish!!.
[True, True, False, False, False]
[True, True, False, False, False]
[True, True, False, False, False]
[True, True, False, Fals

KeyboardInterrupt: 

In [ ]:
n2.round


In [2]:
a = {}
a["a"] = 1
list(a.values())

1
